<h1 style="text-align: center;">Описание подходов для решения задачи сегментации фасадов жилых домов и подсчёта количества окон</h1>

Для решения задачи будем использовать архитектуру Resnet18, хорошо зарекомендовавшуюся себя в задачах семантической сегментации изображений.

1. Характеристики входного изображения (С x W x H) - 1 x 128 x 128 (grayscale)
2. Размеченные данные - (С x W x H) - 2 x 128 x 128 (два класса [0,1], 0 - background, 1 - windows)
3. При достаточной хорошей обучаемости Unet, для определения контуров окон можно использовать The marching squares algorithm. (The marching squares algorithm is a special case of the marching cubes algorithm (Lorensen, William and Harvey E. Cline. Marching Cubes: A High Resolution 3D Surface Construction Algorithm. Computer Graphics SIGGRAPH 87 Proceedings) 21(4) July 1987, p. 163-170).)

The marching squares algorithm возвращает требуемое количество контуров, соответствующих окнам, а также индексы пикселей, соответствующие центрам контуров, чтобы можно было посчитать сетку колонн/рядов окон.

Работа алгоритма:

<image src="https://scikit-image.org/docs/stable/_images/sphx_glr_plot_contours_001.png" alt="Описание картинки">





In [7]:
import os
import shutil

import torch 
import torch.nn as nn

import numpy as np
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torchvision import datasets
from torchsummary import summary
import torchmetrics.classification as tmc
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import cv2 
import albumentations as A
from skimage import measure # measure.find_contours() - The marching squares algorithm
from PIL import Image, ImageOps
from sklearn.utils import compute_class_weight
from sklearn.model_selection import train_test_split
import segmentation_models_pytorch as smp
%matplotlib inline



# Dataset: CMP_facade_DB_base
https://cmp.felk.cvut.cz/~tylecr1/facade/



In [3]:
TARGET_IMAGE_WIDTH = 128
TARGET_IMAGE_HEIGHT = 128
NUM_CLASSES = 2 
NUM_COLORS = 1
BATCH_SIZE = 64
EXECUTION_IMAGE_DATA = True
EXECUTION_MODEL_TRAIN = True

In [4]:
cwd = os.getcwd()
img_trainA_path = os.path.join(cwd, r'dataset\trainA')
img_testA_path = os.path.join(cwd, r'dataset\testA')
img_trainB_path = os.path.join(cwd, r'dataset\trainB')
img_testB_path = os.path.join(cwd, r'dataset\testB')
npy_path = os.path.join(cwd, r'dataset\np.array_targets')
images_path = os.path.join(cwd, r'dataset\images')
masks_path = os.path.join(cwd, r'dataset\masks')

## Data Augmentation

In [39]:
def data_augmentation(img_folder, mask_folder, num_of_augmentaions):

    """image transformations"""
    
    transform = A.Compose([A.VerticalFlip(),
                            A.Rotate(),
                            A.HorizontalFlip(),
                            A.GridDistortion(p=0.33),
                            ])

    tree = os.walk(img_folder)
    cwd = os.getcwd()
    for root, dirs, files in tree:
        for file in files:
            image_base_path = os.path.join(root, file)
            mask_base_path = os.path.join(mask_folder, file) 
            mask_base_path = mask_base_path.replace('.jpg', '.png')
            image_base = np.array(Image.open(image_base_path).convert('RGB'))
            mask_base = np.array(Image.open(mask_base_path).convert('RGB'))
            for i in range(num_of_augmentaions):
                transformed = transform(image=image_base, mask=mask_base)
                transform_image_save_path = image_base_path.replace('.jpg',f'__{i}.jpg')
                transform_mask_save_path = mask_base_path.replace('.png',f'__{i}.png')

                transformed_image = Image.fromarray(transformed['image'])
                transformed_image.save(transform_image_save_path)
                transformed_mask = Image.fromarray(transformed['mask'])
                transformed_mask.save(transform_mask_save_path)


num_of_augs = 15          
#data_augmentation(images_path, masks_path, num_of_augs)

### Train_test_split

In [40]:
def train_test_split_():

    for file1, file2, file3, file4 in zip(os.scandir(img_trainA_path), os.scandir(img_trainB_path),os.scandir(img_testA_path),os.scandir(img_testB_path)):
        os.remove(file1.path)
        os.remove(file2.path)
        os.remove(file3.path)
        os.remove(file4.path)

    masks_list = []
    images_list = []
    images_tree = os.walk(images_path)
    masks_tree = os.walk(masks_path)
    for root, dirs, files in images_tree:
        for filename in files:
            images_list.append(filename)

    for root, dirs, files in masks_tree:
        for filename in files:
            masks_list.append(filename)

    X_train, X_val, y_train, y_val = train_test_split(images_list, masks_list, test_size=0.3)


    for x_t, y_t in zip(X_train, y_train):
        shutil.copy(os.path.join(images_path, x_t), os.path.join(img_trainA_path, x_t))
        shutil.copy(os.path.join(masks_path, y_t), os.path.join(img_trainB_path, y_t))

    for x_val, y_val in zip(X_val, y_val):
        shutil.copy(os.path.join(images_path, x_val), os.path.join(img_testA_path, x_val))
        shutil.copy(os.path.join(masks_path, y_val), os.path.join(img_testB_path, y_val))

#train_test_split_()

# Encode labeled masks to classes 

In [41]:
for file1, file2 in zip(os.scandir(os.path.join(npy_path, r'npy_trainB')), os.scandir(os.path.join(npy_path, r'npy_testB'))):
    os.remove(file1.path)
    os.remove(file2.path)


"""resizing pictures without antialiasing"""

transform_mask = transforms.Resize((TARGET_IMAGE_WIDTH, TARGET_IMAGE_HEIGHT), interpolation=transforms.InterpolationMode.NEAREST_EXACT, antialias=False)


# colormap12_gray = {19 : 0, #background #PIL
#             29 : 1, #facade
#             79: 2, #window
#             129: 3, #door
#             126: 4, #cornice
#             194: 5, #sill
#             210: 6, #balcony
#             226: 7, #blind
#             176: 8, #deco
#             179: 9, #molding
#             76: 10, #pillar
#             51: 11 #shop 
#                               }

colormap2_gray = {   19 : 0, #background
                    79: 1 #window    
                                }
                                

def to_categorical(y: np.array, num_classes: int) -> np.array:
    """one-hot encoding"""
    return np.eye(num_classes, dtype='uint8')[y.astype('uint8')]

def encode_binary_gray(img_array: np.array, colormap):

    """2 class segmentation encoding for sigmoid activation"""
    class_array = np.zeros((img_array.shape[0], img_array.shape[1]))
    image_trans_array = np.zeros((img_array.shape[0], img_array.shape[1]))

    for W in range(img_array.shape[0]):
        for H in range(img_array.shape[1]):
            class_array[W,H] = colormap.get(img_array[W,H], 0)
            gray_color = [k for k,v in colormap.items() if v==class_array[W,H]][0]
            image_trans_array[W,H] = gray_color
    return class_array.astype(np.uint8), image_trans_array.astype(np.uint8)


def encode_to_classes2(img_array: np.array, colormap):
    """2 class segmentation encoding for softmax activation"""
    class_array = np.zeros((img_array.shape[0], img_array.shape[1]))
    image_trans_array = np.zeros((img_array.shape[0], img_array.shape[1]))

    for W in range(img_array.shape[0]):
        for H in range(img_array.shape[1]):
            #pixel_color = tuple(img_array[W,H])
            class_array[W,H] = colormap.get(img_array[W,H], 0)
            image_trans_array[W,H] = img_array[W,H]
    class_array = to_categorical(class_array,2)
    return class_array.astype(np.uint8), image_trans_array.astype(np.uint8)
    

def create_image_data(execute) -> np.array:
    """saving encoded class arrays as .npy on disk"""
    if execute:
        tree = os.walk(os.path.join(cwd, r'dataset\trainB'))
        for root, dirs, files in tree:
            for filename in files:
                mask_filename = os.path.join(root, filename)
                mask = Image.open(mask_filename)
                mask = ImageOps.grayscale(mask)
                mask = transform_mask(mask)
                mask = np.array(mask)
                if NUM_CLASSES == 2:
                    class_array, img_trans_array = encode_to_classes2(mask, colormap2_gray)
                else:
                    class_array, img_trans_array = encode_binary_gray(mask, colormap2_gray)
                    #Image.fromarray(img_trans_array).save(os.path.join(r'D:\Coding\test_case_cv\Unet\dataset\check_masks', filename))
                np.save(os.path.join(npy_path, r'npy_trainB', filename.replace('.png', '.npy')), class_array)

        tree = os.walk(os.path.join(cwd, r'dataset\testB'))
        for root, dirs, files in tree:
            for filename in files:
                mask_filename = os.path.join(root, filename)
                mask = Image.open(mask_filename)
                mask = ImageOps.grayscale(mask)
                mask = transform_mask(mask)
                mask = np.array(mask)
                if NUM_CLASSES == 2:
                    class_array, img_trans_array = encode_to_classes2(mask, colormap2_gray)
                else:
                    class_array, img_trans_array = encode_binary_gray(mask, colormap2_gray)
                    
                np.save(os.path.join(npy_path, r'npy_testB', filename.replace('.png', '.npy')), class_array)
                
#create_image_data(EXECUTION_IMAGE_DATA)

d:\Coding\test_case_cv\Unet\venv\lib\site-packages\torchvision\transforms\functional.py:488: UserWarning: Anti-alias option is always applied for PIL Image input. Argument antialias is ignored.
  warnings.warn("Anti-alias option is always applied for PIL Image input. Argument antialias is ignored.")


In [42]:
"""Balancing weights for loss function"""

# tree = os.walk(os.path.join(cwd, r'dataset\np.array_targets\npy_trainB'))
# mask_list = []
# for root, dirs, files in tree:
#             for filename in files:
#                 mask_filename = os.path.join(root, filename)
#                 mask_list.append(np.load(mask_filename))

# tree = os.walk(os.path.join(cwd, r'dataset\np.array_targets\npy_testB'))
# mask_list = []
# for root, dirs, files in tree:
#             for filename in files:
#                 mask_filename = os.path.join(root, filename)
#                 mask_list.append(np.load(mask_filename))

# masks_encoded = np.array(mask_list)
# masks_reshaped_encoded = masks_encoded.reshape(-1,1).flatten()

# # class_weights = compute_class_weight(class_weight='balanced', 
# #                                     classes = np.unique(masks_reshaped_encoded), 
# #                                     y=masks_reshaped_encoded)


KeyboardInterrupt: 

In [43]:
#  class_weights = torch.Tensor(class_weights).to('cuda')
#  print(class_weights)

In [44]:
def normalize2_gray():
    tree_train = os.walk(img_trainA_path)
    tree_test = os.walk(img_testA_path)

    all_images_pathnames = []
    for root,dirs, files in tree_train:
        for filename in files:
            all_images_pathnames.append(os.path.join(root,filename))

    for root,dirs, files in tree_test:
        for filename in files:
            all_images_pathnames.append(os.path.join(root,filename))
    
    mean = np.array([0.])
    stdTemp = np.array([0.])
    std = np.array([0.])
    
    numSamples = len(all_images_pathnames)
    for i in range(numSamples):
        image = Image.open(all_images_pathnames[i])
        image= ImageOps.grayscale(image)
        image = np.array(image).astype(np.float32) / 255
        mean[0] += np.mean(image)
        
    mean_gray = mean[0] / numSamples
    mean_gray = [mean_gray]

    for i in range(numSamples):
        image = Image.open(all_images_pathnames[i])
        image= ImageOps.grayscale(image)
        image = np.array(image).astype(np.float32) / 255
        stdTemp[0] += (image - mean_gray[0]**2).sum()/(image.shape[0]*image.shape[1])
    
    std = np.sqrt(stdTemp/numSamples)
    std_gray=[std[0]]
    print(mean_gray, std_gray)
    return mean_gray, std_gray


mean = [0.45470184212915166]
std = [0.49794385661480856]

#mean, std = normalize2_gray()


# Custom Dataset class and instances #


In [46]:
transform_image = transforms.Compose(
    [   transforms.Resize((TARGET_IMAGE_WIDTH, TARGET_IMAGE_HEIGHT)),
        transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
    ]
)
transform_label = transforms.ToTensor() # unused

image_train_ids = [] # pass to Pytorch Dataset class
image_train_path_list = [] # pass to Pytorch Dataset class
mask_train_path_list = [] # pass to Pytorch Dataset class

image_test_ids = [] # pass to Pytorch Dataset class
image_test_path_list = [] # pass to Pytorch Dataset class
mask_test_path_list = [] # pass to Pytorch Dataset class

# get python lists with image and maskspaths
tree_train_img = os.walk(os.path.join(cwd, r'dataset\trainA')) 
for root, dirs, files in tree_train_img:
            for filename in files:
                img_id = filename.replace('.jpg','')
                image_path = os.path.join(root, filename)
                image_train_ids.append(img_id)
                image_train_path_list.append(os.path.join(root, filename))


tree_test_img = os.walk(os.path.join(cwd, r'dataset\testA')) 
for root, dirs, files in tree_test_img:
            for filename in files:
                img_id = filename.replace('.jpg','')
                image_path = os.path.join(root, filename)
                image_test_ids.append(img_id)
                image_test_path_list.append(os.path.join(root, filename))


tree_train_npy = os.walk(os.path.join(npy_path, r'npy_trainB'))
for root, dirs, files in tree_train_npy:
            for filename in files:
                mask_train_path_list.append(os.path.join(root, filename))

tree_test_npy = os.walk(os.path.join(npy_path, r'npy_testB'))
for root, dirs, files in tree_test_npy:
            for filename in files:
                mask_test_path_list.append(os.path.join(root, filename))

class CustomImageDataset(Dataset):
    """Custom dataset class for pytorch dataloader"""
    def __init__(self, ids_list, image_path_list, mask_path_list, transform_image, transform_label):
        self.image_ids = ids_list
        self.image_path_list = image_path_list
        self.mask_path_list = mask_path_list
        self.transform_image = transform_image
        self.transform_label = transform_label

    def __len__(self):
        return len(self.image_path_list)

    def __getitem__(self, idx):
        img_path = self.image_path_list[idx]
        img_label_path = self.mask_path_list[idx]
        image = Image.open(self.image_path_list[idx])
        image= ImageOps.grayscale(image)
        image_label = np.load(self.mask_path_list[idx])
        text_label = self.image_ids[idx]
        if self.transform_image:
            image = self.transform_image(image)
        if self.transform_label:
            image_label = torch.Tensor(image_label)
            image_label = torch.permute(image_label, dims=(2,0,1))
        return image, image_label, text_label


training_dataset = CustomImageDataset(ids_list=image_train_ids,
                                        image_path_list=image_train_path_list,
                                        mask_path_list=mask_train_path_list,
                                   transform_image=transform_image,
                                   transform_label=transform_label,
                                   )

validation_dataset = CustomImageDataset(ids_list=image_test_ids,
                                        image_path_list=image_test_path_list,
                                        mask_path_list=mask_test_path_list,
                                   transform_image=transform_image,
                                   transform_label=transform_label,
                                   )

torch.Size([2, 128, 128])

# DataLoader instances #

In [47]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_dataset, batch_size=BATCH_SIZE, pin_memory=(torch.cuda.is_available()), shuffle=True, drop_last=False)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, pin_memory=(torch.cuda.is_available()), shuffle=True, drop_last=False)


# Unet Model implementation #

In [48]:
class conv_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.conv1 = nn.Conv2d(in_c, out_c, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_c)         
        self.conv2 = nn.Conv2d(out_c, out_c, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_c)         
        self.relu = nn.ReLU()  
        self.dropout = nn.Dropout(p=0.3)

        
    def forward(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu(x)        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)    
        return x

class encoder_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.conv = conv_block(in_c, out_c)
        self.pool = nn.MaxPool2d((2, 2))
        self.dropout = nn.Dropout(p=0.3) 

    def forward(self, inputs):
        x = self.conv(inputs)
        p = self.pool(x)
        return x, p

class decoder_block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_c, out_c, kernel_size=2, stride=2, padding=0)
        self.conv = conv_block(out_c+out_c, out_c)    

    def forward(self, inputs, skip):
        x = self.up(inputs)
        x = torch.cat([x, skip], axis=1)
        x = self.conv(x)
        return x

class Unet(nn.Module):
    def __init__(self, num_channels, num_of_colors):
        super().__init__() 
        """ Encoder """
        self.e1 = encoder_block(num_of_colors, 64)
        self.e2 = encoder_block(64, 128)
        self.e3 = encoder_block(128, 256)
        self.e4 = encoder_block(256, 512)        
        """ Bottleneck """
        self.b = conv_block(512, 1024)         
        """ Decoder """
        self.d1 = decoder_block(1024, 512)
        self.d2 = decoder_block(512, 256)
        self.d3 = decoder_block(256, 128)
        self.d4 = decoder_block(128, 64)        
        """ Classifier """
        self.sigmoid = nn.Sigmoid()
        self.outputs = nn.Conv2d(64, num_channels, kernel_size=1, padding=0)     
        
    def forward(self, inputs):
        """ Encoder """
        s1, p1 = self.e1(inputs)
        s2, p2 = self.e2(p1)
        s3, p3 = self.e3(p2)
        s4, p4 = self.e4(p3)         
        """ Bottleneck """
        b = self.b(p4)         
        """ Decoder """
        d1 = self.d1(b, s4)
        d2 = self.d2(d1, s3)
        d3 = self.d3(d2, s2)
        d4 = self.d4(d3, s1)         
        """ Classifier """
        outputs = self.outputs(d4) 
        return outputs

# Prepare and run model #

### Accuracy metrics

In [30]:
def get_acc(preds_batch, labels_batch):
    N = TARGET_IMAGE_HEIGHT * TARGET_IMAGE_HEIGHT
    preds = torch.softmax(preds_batch.squeeze(0), dim=0)
    preds = torch.argmax(preds, dim=0).to(torch.int32)
    labels = torch.argmax(labels_batch.squeeze(0), dim=0).to(torch.int32)
    return torch.sum(torch.eq(preds,labels)) / N

### Model parameters

In [50]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

LEARNING_RATE = 0.001
EPOCHS =250
writer_step = 2
WEIGHT_DECAY = 0


model =smp.Unet(encoder_name='resnet18', 
                        encoder_weights='imagenet',
                        in_channels=1,
                        classes=2).to(device)
        
# model = Unet(num_channels=NUM_CLASSES, num_of_colors=NUM_COLORS).to(device)
# print(summary(model))
loss_fn = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), 
                                lr=LEARNING_RATE, 
                                #weight_decay=WEIGHT_DECAY, 
                                )


def main(execute):
    if execute:
        def train_one_epoch(epoch_index, tb_writer):
            running_loss = 0
            last_loss = 0
            running_acc = 0
            last_acc = 0
            for i, batch in enumerate(train_dataloader):
                inputs, labels, _ = batch
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                preds = model(inputs)
                loss = loss_fn(preds, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                running_acc = get_acc(preds, labels)
                last_acc = running_acc
                last_loss = running_loss
                print('  batch {} loss: {}'.format(i + 1, last_loss))
                tb_x = epoch_index * len(train_dataloader) + i + 1
                tb_writer.add_scalars('Training Accuracy/Loss', {'Training Loss': last_loss, 'Training Accuracy': last_acc*100}, tb_x) 
                running_loss = 0.
                running_acc = 0.
                #scheduler.step(last_loss)
            return last_loss

        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
        epoch_number = 0


        for epoch in range(EPOCHS):
            print('EPOCH {}:'.format(epoch_number + 1))

            model.train(True)
            avg_loss = train_one_epoch(epoch_number, writer)

            model.train(False)
            with torch.no_grad():
                running_vloss = 0.0
                for i, vdata in enumerate(validation_dataloader):
                    vinputs, vlabels, _ = vdata
                    vinputs, vlabels = vinputs.to(device), vlabels.to(device)
                    voutputs = model(vinputs)
                    vloss = loss_fn(voutputs, vlabels)
                    running_vloss += vloss
                    avg_vloss = running_vloss / (i + 1)
                    
                    running_acc = get_acc(voutputs, vlabels)
                    avg_acc = running_acc / (i + 1)
                    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))
                    print(f'Validation accuracy = {avg_acc*100} %')
                    # Log the running loss averaged per batch
                    # for both training and validation
                    writer.add_scalars('Validation Loss/Accuracy',
                                    { 'Validation Loss' : avg_loss, 'Validation Accuracy' : avg_acc*100},
                                    epoch_number + 1)
                    writer.flush()
                
            epoch_number += 1


        writer.close()

        tree = os.walk(cwd, topdown=True)

        for root, dirs, files in tree:
            for name in files:
                if '.pth' in os.path.join(cwd, name):
                    os.remove(os.path.join(cwd, name))
                    break
            break

        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path + '.pth')

main(EXECUTION_MODEL_TRAIN)

cuda
EPOCH 1:
  batch 1 loss: 0.9395301938056946
  batch 2 loss: 0.808032751083374
  batch 3 loss: 0.6969606876373291
  batch 4 loss: 0.6363489031791687
  batch 5 loss: 0.5753549337387085
  batch 6 loss: 0.5363470911979675
  batch 7 loss: 0.4965018630027771
  batch 8 loss: 0.4739192724227905
  batch 9 loss: 0.45043110847473145
  batch 10 loss: 0.4240991175174713
  batch 11 loss: 0.40859001874923706
  batch 12 loss: 0.37980687618255615
  batch 13 loss: 0.3762441575527191
  batch 14 loss: 0.357334166765213
  batch 15 loss: 0.3528655767440796
  batch 16 loss: 0.3413981795310974
  batch 17 loss: 0.3198183476924896
  batch 18 loss: 0.3368654251098633
  batch 19 loss: 0.31707096099853516
  batch 20 loss: 0.31928908824920654
  batch 21 loss: 0.2894871234893799
  batch 22 loss: 0.3048691749572754
  batch 23 loss: 0.2903040647506714
  batch 24 loss: 0.3055132031440735
  batch 25 loss: 0.2766028940677643
  batch 26 loss: 0.2870015501976013
  batch 27 loss: 0.27847447991371155
  batch 28 loss: 0.

# Pre trained Unet encoders

# Make Predictions on single Image #

In [ ]:
cwd = os.getcwd()


tree = os.walk(cwd, topdown=True)
for root, dirs, files in tree:
    for name in files:
        if '.pth' in os.path.join(cwd, name):
            model_path = os.path.join(cwd, name)
            break
    break

model =smp.Unet(encoder_name='resnet50', 
                encoder_weights='imagenet',
                in_channels=1,
                classes=2).to(device)

# model = Unet(num_channels=NUM_CLASSES, num_of_colors=NUM_COLORS).to(device)

model.load_state_dict(torch.load(model_path))


def decode_to_RGB2(model, image_path, save_path, colormap): #gray
    """segmentation predicton for 2 class segmentation"""
    image_test_path = os.path.join(cwd, image_path) 
    image_test = Image.open(image_test_path)
    image_test = ImageOps.grayscale(image_test)
    image_test = transform_image(image_test)
    image_test = image_test.clone().detach().to(device)
    image_test .to(device)

    output = model(image_test .unsqueeze(0)).squeeze(0)
    output = torch.permute(output, (1,2,0)) #swap axes of tensor
    output = torch.argmax(output, dim=2)
    image_classes = output.cpu().detach().numpy()
    image_output = np.zeros(shape=(TARGET_IMAGE_WIDTH,TARGET_IMAGE_HEIGHT)).astype(np.uint8)
    for W in range(image_classes.shape[0]):
        for H in range(image_classes.shape[1]):
            image_output[W,H] = [k for k, v in colormap.items() if v == image_classes[W,H]][0]
    image_output = Image.fromarray(image_output.astype(np.uint8))
    image_output.save(save_path)


def decode_binary_gray(model, image_path, save_path, colormap):
    image_test_path = os.path.join(cwd, image_path) 
    image_test = Image.open(image_test_path)
    image_test = ImageOps.grayscale(image_test)
    image_test = transform_image(image_test)
    image_test = image_test.clone().detach().to(device)

    output = model(image_test .unsqueeze(0))
    output = torch.sigmoid(output).squeeze(0)
    output = torch.permute(output, dims=(1,2,0))

    image_classes = output.cpu().detach().numpy()
    image_output = np.zeros((TARGET_IMAGE_WIDTH, TARGET_IMAGE_HEIGHT))
    for W in range(image_classes.shape[0]):
        for H in range(image_classes.shape[1]):
            print(image_classes[W,H])
            if image_classes[W,H] >0.5:
                image_output[W,H] = 79
            else:
                image_output[W,H] = 19

    image_output = Image.fromarray(image_output.astype(np.uint8)).save(save_path)

if NUM_CLASSES == 2:
    decode_to_RGB2(model, 'test_image_validation.jpg', 'test_mask_validation_predicted.png', colormap2_gray)
    decode_to_RGB2(model, 'test_image_train.jpg', 'train_mask_validation_predicted.png', colormap2_gray)
elif NUM_CLASSES == 1:
    decode_binary_gray(model, 'test_image_validation.jpg', 'test_mask_validation_predicted.png', colormap2_gray)
    decode_binary_gray(model, 'test_image_train.jpg', 'train_mask_validation_predicted.png', colormap2_gray)



# Count number of windows #

In [ ]:
"""The marching squares algorithm for counting windows"""

test_image_path = os.path.join(cwd, 'test_mask_predicted.png')

def count_windows(test_img_path):
    if os.path.exists(test_image_path):
        imgray = cv2.imread(test_image_path, cv2.IMREAD_GRAYSCALE)
        imgrgb = cv2.imread(test_image_path)

        # get contours
        contours = measure.find_contours(imgray)

        # get contours length
        contour_length_list = []
        for contour in contours:
            contour_length = 0
            for i in range(len(contour)-1):
                contour_length += cv2.norm(contour[i], contour[i+1], cv2.NORM_L2)
            contour_length_list.append(contour_length)

        max_countour_length = max(contour_length_list)
        # remove countours with length less than 20% of mean length of all countours
        windows_count = len([length for length in contour_length_list if length > max_countour_length/3])
        return windows_count

windows = count_windows(test_image_path)
print(f"Количество окон - {windows}")


Количество окон - None
